## Decorador (*Decorator*)

__Decorador__ es un patron de tipo Estructural. Nos permite "envolver" un
objeto, método o función, que realiza una determinada función, con otro objeto,
método o función que copia su interfaz; es decir, que para el resto del mundo se
comporta y se puede interactuar con el exactamente igual que si fuera el
original. De esta forma se puede modificar el comportamiento, normalmente para
añadir funcionalidad.

La razón del nombre de este componente viene de uno de sus primeros ejemplos de
uso, en los sistemas de ventanas. En estos sistemas, el contenido visual de una
aplicación no tiene bordes, ni botones, ni controles como barras de
desplazamiento, etc... Es solo un cuadrado con el contenido de la aplicación. Se
aplicaban una función (se decoraba) al componente, de forma que, por ejemplo,
ante una operación de dibujar la ventana, digamos `redraw`, el decorador pintaba
los bordes, los botones, etc. y luego llamaba a la operación `redraw` de la
aplicación, que pintaba los contenidos.

Eso permitia cambiar el sistema de ventanas para tener estilos totalmente
diferentes, solo cambiando el decorador que "envuelve" a la aplicación. Incluso
si eliminabas el decorador, el sistema seguía funcionando (Sin bordes, claro)
porque al sistema de ventanas solo le interesaba que los componentes tengan un
método `redraw` (la interfaz es la misma).

Las razones para usar este patrón son normalmente dos:

- Mejorar la respuesta de un componente a otro componente

- Proporcionar multiples comportamientos opcionales

Se utiliza a menudo la segunda opción como una alternativa a la herencia
múltiple. Se puede crear un objeto básico, y luego "envolverlo" con un
decorador. Como la interfaz del decorador es igual que la del objeto base,
podemos usar diferentes decoradores para el mismo objeto o incluso anidar
decoradores, es decir, decorando un objeto ya decorado.


## Un ejemplo de decorador.

Usaremos un ejemplo de decorador a la vez que vemos algo de programación de redes. En este
caso usaremos un *socket* TCP. EL método `send` de los *sockets* acepta como parámetro
una secuencia de bytes y los envia por la conexión establecida hacie el otro extremo. Hay muchas
librerías de sockets pero python viene con una ya incorporada en la libreria estándar.

Vamos a crear en primer lugar un servidor que espere a que algun otro proceso se conecte; en
ese momento pregunta un texto el usuario, y devuelve ese texto como respuesta.

In [ ]:
# %load socket-server.py
#!/usr/bin/env python

import socket

def respond(client, addr):
    response = input("Enter a value: ")
    message = f"Hello {addr} this is your response: {response}"
    client.send(message.encode('utf8'))
    client.close()
    return response == 'exit'

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(('localhost', 8401))
server.listen(1)
try:
    while True:
        client, addr = server.accept()
        must_exit = respond(client, addr)
        if must_exit:
            break
finally:
    server.close()

Ahora, antes de escribir el cliente, un aviso: para poder ejecutar la cliente y el servicor, estos
tienen que ejecutarse como procesos distintos, asi que hay que ejecutar el servidor en un notebook, con
su propio kernel, y el cliente en otro. Hay dos notebooks ya preparados para este proposito
[socket-server.ipynb](./socket-server.ipynb) y [socket-client.ipynb](./socket-client.ipynb)

In [ ]:
!cat socket-client.py

Para usar estos programas:
    
❶ Arrancar el programa `socket-server.py`, ya sea en una terminal o abriendo en
una pestaña nueva del navegador con [`socket-server.ipynb`](./socket-server.ipynb).

❷ Arrancar el programa `socket-client.py`, ya sea en una terminal o abriendo en
una pestaña nueva del navegador con [`socket-client.ipynb`](./socket-client.ipynb).

❸ En la pestaña o terminal del servidor veremos que se pregunta al usuario
por el texto de la respuesta; escribir algo y pulsar enter.

❹ En la pestaña / terminal del cliente, veremos que hemos recibido una
respuesta con el texto que metimos.

Puedes ejecutar el cliente las veces que quieras. El cliente enviará los textos
que se introduzcan en el servidor.

Ahora, revisando nuestro codigo servidor, vamos a añadir un par de decoradores para 
el socket. El primero sera un decorador de *Logging*. Este decorador simplemente
imprime en la consola del servidor los datos enviados.

In [ ]:
class LogSocket:

    def __init__(self, socket):
        self.socket = socket

    def send(self, data):
        print("Sending {0} to {1}".format(
            data, self.socket.getpeername()[0]))
        self.socket.send(data)

    def close(self):
        self.socket.close()

El decorador mantiene la misma interfaz que el onjeto socket, asi que, a los
efectos de la función `respond`, lo mismo le da usar un socket normal
o nuestra versión decorada. El decorador debería implementar *toda* la interfaz
de los sockets, no solo los métodos `send` y `close`. Tambien debería implementar
mejor el método `send`, ya que este acepta un argumento opcional del que nosotros
no no ocupamos, pero lo hemos hecho así para mantener el ejemplo lo más sencillo
posible.


### Ejercicio: Modificar el código del servidor para que use nuestra nueva clase decorada

**PISTA**: Solo hay que cambiar una línea del código original para usar nuestra clase
derivada; en vez de llamar a la función `respond` con el socket, se le llama con 
nuestro *socket* decorado, creado a parir del *socket* original.

In [ ]:
import socket

class LogSocket:

    def __init__(self, socket):
        self.socket = socket

    def send(self, data):
        print("Sending {0} to {1}".format(
            data, self.socket.getpeername()[0]))
        self.socket.send(data)

    def close(self):
        self.socket.close()


def respond(client, addr):
    response = input("Enter a value: ")
    message = f"Hello {addr} this is your response: {response}"
    client.send(message.encode('utf8'))
    client.close()
    return response == 'exit'

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(('localhost', 8401))
server.listen(1)
try:
    while True:
        client, addr = server.accept()
        must_exit = respond(LogSocket(client), addr)
        if must_exit:
            break
finally:
    server.close()

Esta solución es simple, pero desde el punto de vista de la programación
orientada a objetos habría otra opcion, crear nuestra propia clase,
derivada de *socket*, y sobreescribir el método `send` (Llamanda al
método `send` original con `super().send(...)`, despues de imprimir
el mensaje a la consola).

Esta opción también es perfectamente válida, para este caso. La ventaja que
ofrece el decorador es que nos permite modificar el comportamiento
dinámicamente (Algo que no pasa en nuestro ejemplo, pero que puede
ser deseable en otros acasos).

Para lenguajes que no soportan herencia múltiple, también se usa este patrón 
para asignar varios comportamientos a un método.

Vamos a ver un ejemplo de esto creando un segundo decorador. Este decorador
lo que va a hacer es comprimir el contenido a enviar usando la librería
estándar `zlib`.

In [ ]:
import zlib

class GzipSocket:

    def __init__(self, socket):
        self._socket = socket

    def send(self, data):
        c_data = zlib.compress(data)
        self._socket.send(c_data)

    def close(self):
        self._socket.close()

El método `send` en este caso comprime los datos antes de enviarlos al cliente.

Ahora, con este decorador, vamos a escribir una nueva versión del
servidor que, de forma dinámica, toma la decisión de si comprimir o no
los datos de respuesta. Podemos imaginar que de alguna manera el cliente
ha informado al servidor de si es capaz de aceptar comtenidos comprimidos o no.

Para mantener el ejemplo sencillo, vamos a suponer que hay un objeto `options`
con un valor lógico `True` o `False`, llamada `use_compress`. El código podría
quedar más o menos así:

In [1]:
import socket
import zlib

class ZCompressSocket:

    def __init__(self, socket):
        self._socket = socket

    def send(self, data):
        c_data = zlib.compress(data)
        self._socket.send(c_data)

    def close(self):
        self._socket.close()


def respond(client, addr):
    response = input("Enter a value: ")
    message = f"Hello {addr} this is your response: {response}"
    client.send(message.encode('utf8'))
    client.close()
    return response == 'exit'


server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind(('localhost', 8401))
server.listen(1)
try:
    while True:
        client, addr = server.accept()
        must_exit = respond(ZCompressSocket(client), addr)
        if must_exit:
            break
finally:
    server.close()

OSError: [Errno 48] Address already in use

Copy
        client, addr = server.accept()
        if log_send:
            client = LoggingSocket(client)
        if client.getpeername()[0] in compress_hosts:
            client = GzipSocket(client)
        respond(client)
This code checks a hypothetical configuration variable named log_send. If it's enabled, it wraps the socket in a LoggingSocket decorator. Similarly, it checks whether the client that has connected is in a list of addresses known to accept compressed content. If so, it wraps the client in a GzipSocket decorator. Notice that none, either, or both of the decorators may be enabled, depending on the configuration and connecting client. Try writing this using multiple inheritance and see how confused you get!

### Decorators in Python

El patrón decorador es muy usado en Python, hasta el punto que se proporciona una
sintaxis especial para decorar funciones, métodos e incluso clases. 

Por ejemplo, podemos generalizar nuestro decorador de *logging*.
Podria ser muy útil no solo para nuestro ejemplo con los *sockets*, sino
con cualquier función. El siguiente ejemplo implemente este decorador
más general:

In [8]:
import time

def logged(func):
    
    def wrapper(*args, **kwargs):
        name = func.__name__
        start_time = time.time()
        print(f"Ejecutando {name} con argumentos {args} y {kwargs}")
        return_value = func(*args, **kwargs)
        delta = time.time() - start_time
        print(f"Executado {name} en {delta} milisegundos")
        print(f"El resultado fue {return_value}")
        return return_value
    
    return wrapper

def test1(a,b,c):
    print("\ttest1 called")

def test2(a,b):
    print("\ttest2 called")
    return 7

def test3(a,b):
    print("\ttest3 called")
    time.sleep(1)

test1 = logged(test1)
test2 = logged(test2)
test3 = logged(test3)

test1(1,2,3)
test2(4,b=5)
test3(6,7)

Ejecutando test1 con argumentos (1, 2, 3) y {}
	test1 called
Executado test1 en 0.00036263465881347656 milisegundos
El resultado fue None
Ejecutando test2 con argumentos (4,) y {'b': 5}
	test2 called
Executado test2 en 0.0001780986785888672 milisegundos
El resultado fue 7
Ejecutando test3 con argumentos (6, 7) y {}
	test3 called
Executado test3 en 1.001971960067749 milisegundos
El resultado fue None


Este decorador es conceptualmente igual que el del ejemplo anterior; allí
el decorador acepta algo que parece un socket y devolvia algo que también
parece un socket. En este caso, este decorador acepta como parametro
una función y devuelve una nueva función. Este proceso se realiza en tres
pasos:

- En primer lugar tenemos la funcion `logged`, que acepta como parámetro
  de entrada otra función.
  
- Internamente, se define una función (En nuestro ejemplo, *wrapper*), que
  hace una serie de cosas antes y despues de llamar a la función
  original
  
- Se devuelve la nueva función.